# Julia 機器學習：GLM 線性迴歸

## 作業 027：波士頓房價預測資料集

請使用 GLM 中的模型，建立一個預測模型來預測波士頓的房價。

In [1]:
using GLM, RDatasets, MLDataUtils

## 讀取資料

In [2]:
boston = dataset("MASS", "Boston")
first(boston, 10)

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.09,1,296
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242
3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242
4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222
5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222
6,0.02985,0.0,2.18,0,0.458,6.43,58.7,6.0622,3,222
7,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311
8,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311
9,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311


In [3]:
indecies = MLDataUtils.shuffleobs(collect(1:nrow(boston)))
train_ind, test_ind = MLDataUtils.splitobs(indecies, at = 0.8);

In [4]:
train = boston[train_ind, :]
test = boston[test_ind, :]

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,0.09068,45.0,3.44,0,0.437,6.951,21.5,6.4798,5,398
2,0.22489,12.5,7.87,0,0.524,6.377,94.3,6.3467,5,311
3,0.22927,0.0,6.91,0,0.448,6.03,85.5,5.6894,3,233
4,1.35472,0.0,8.14,0,0.538,6.072,100.0,4.175,4,307
5,5.29305,0.0,18.1,0,0.7,6.051,82.5,2.1678,24,666
6,3.47428,0.0,18.1,1,0.718,8.78,82.9,1.9047,24,666
7,7.02259,0.0,18.1,0,0.718,6.006,95.3,1.8746,24,666
8,0.16439,22.0,5.86,0,0.431,6.433,49.1,7.8265,7,330
9,13.5222,0.0,18.1,0,0.631,3.863,100.0,1.5106,24,666


In [19]:
ols = GLM.lm(@formula(MedV ~ Crim + Zn + Indus + Chas + NOx + Rm + Age + Dis + Rad + Tax), train)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

MedV ~ 1 + Crim + Zn + Indus + Chas + NOx + Rm + Age + Dis + Rad + Tax

Coefficients:
──────────────────────────────────────────────────────────────────────────────────
               Estimate  Std. Error    t value  Pr(>|t|)    Lower 95%    Upper 95%
──────────────────────────────────────────────────────────────────────────────────
(Intercept)  -1.82       4.54381     -0.400545    0.6890  -10.7531      7.11314   
Crim         -0.184107   0.0380044   -4.84437     <1e-5    -0.258824   -0.109391  
Zn            0.0697046  0.0164439    4.23894     <1e-4     0.0373759   0.102033  
Indus        -0.105978   0.0756443   -1.401       0.1620   -0.254695    0.0427394 
Chas          3.78119    1.17029      3.23097     0.0013    1.48038     6.08199   
NOx          -8.60816    4.42228     -1.94654     0.0523  -17.3024      0.086056

In [15]:
pred=predict(ols, test)

101-element Array{Union{Missing, Float64},1}:
 29.59360160284755  
 19.258549199798644 
 19.018978137615964 
 19.123949305663153 
 18.138944117156864 
 41.65002533558521  
 17.119535858769254 
 21.418583525075828 
  1.9901440430095025
 17.165893790304626 
 18.638213475706834 
 20.945350064744304 
 35.76061190944511  
  ⋮                 
 18.307948852789814 
 22.595388979512588 
 27.736310673592538 
 20.265781567970194 
 15.599384378872479 
 24.616674007261096 
 14.64898234382153  
 25.74019557635696  
 31.308302414710226 
 23.809820949712325 
 18.232647370619663 
 18.143863988628752 

In [16]:
GLM.r²(ols)

0.6481951341583051

In [17]:
GLM.adjr²(ols)

0.6392660766496326